In [ ]:
%load_ext autoreload
%autoreload 2

from utility_script import *

import os
import re
import json
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
mymodel = MyBERT(train=True)
mymodel.compile()

In [ ]:
data = Data()
train_df = data.read_train_from_file() #nrows= 100
print('train_df shape before ', train_df.shape)
train_df_1 = train_df.loc[train_df['dataset_found']].groupby(['dataset_name', 'section_title']).head(500).reset_index(drop=True)
train_df_2 = train_df.loc[~train_df['dataset_found']].sample(frac=1)
print(train_df_1.shape)
print(train_df_2.shape)
train_df = train_df_1.append(train_df_2)
train_df = train_df.sample(frac=1).reset_index(drop=True)
print('train_df shape after dropping shit ', train_df.shape)
train_df.head(2)

In [ ]:
def train_model(chunk):
    train_encodings = mymodel.fast_tokenizer(chunk[data.lines_col].tolist(), max_length=MAX_LENGTH_PER_SENTENCE, padding=True, truncation=True, return_offsets_mapping=True,is_split_into_words=True, return_tensors='np') #return_offsets_mapping=True # ,
    train_labels = data.encode_tags(chunk[data.labels_col], train_encodings)
    mymodel.train_model(train_encodings, tf.convert_to_tensor(np.array(train_labels)), epochs = 2)


def train_model_large_df(df, iterations = None):
    df.dropna(inplace=True)
    i = 0
    exception_count = 0
    for chunk in np.array_split(df, int(df.shape[0]/128)):
        try:
            train_model(chunk)
        except:
            exception_count += 1
            pass
    print("total exception_count", exception_count)
    #     op = mymodel.model(train_encodings)
    #     get_dataset_names_from_outputs(op)
#         break
#         i += 1
#         if i > 2:
#             break

In [ ]:
%%time
MAX_LENGTH_PER_SENTENCE=256
train_model_large_df(train_df)
# val_labels = encode_tags(val_tags, val_encodings)

In [ ]:
# n = 4877
# test_example = [chunk['lines'].loc[n]]
# test_labels = [chunk['labels'].loc[n]]
# train_encodings = mymodel.fast_tokenizer(test_example, max_length=MAX_LENGTH_PER_SENTENCE, padding=True, truncation=True, return_offsets_mapping=True,is_split_into_words=True, return_tensors='np') #return_offsets_mapping=True # ,
# train_labels = data.encode_tags(test_labels, train_encodings)
# mymodel.train_model(train_encodings, tf.convert_to_tensor(np.array(train_labels)), epochs = 2)

In [ ]:
mymodel.model.save_pretrained("model_file")
mymodel.tokenizer.save_pretrained("tokenizer_file")
# mymodel.fast_tokenizer.save_pretrained("fast_tokenizer_file")

In [ ]:
!ls /kaggle/working/model_file

In [ ]:
#TODO:
# in data class, take from class 0 only if length > 3

In [ ]:
train_df.columns

In [ ]:
train_df.head(2)

In [ ]:
train_df.shape

In [ ]:
train_df.loc[train_df['dataset_found']].groupby('section_title').count().sort_values(by='lines',ascending=False)

In [ ]:
# train_df.loc[train_df['dataset_found']]['lines'].apply(lambda x: len(x)).value_counts()